In [1]:
import tensorflow as tf
import keras
from keras.models import model_from_json 
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


In [2]:
# opening and store file in a variable
json_file = open('model\model.json','r')
loaded_model_json = json_file.read()
json_file.close()

In [3]:
# use Keras model_from_json to make a loaded model
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

In [4]:
# load weights into new model
loaded_model.load_weights("model\model.h5")
print("Awesome, your model has been loaded from disk! Cool beans!")

# compile and evaluate loaded model
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Awesome, your model has been loaded from disk! Cool beans!


In [5]:
ufc = pd.read_csv("resources/modelval_data.csv")
ufc.head()

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,B__Round1_Grappling_Reversals_Landed,...,R__Round5_TIP_Ground Time,R__Round5_TIP_Guard Control Time,R__Round5_TIP_Half Guard Control Time,R__Round5_TIP_Misc. Ground Control Time,R__Round5_TIP_Mount Control Time,R__Round5_TIP_Neutral Time,R__Round5_TIP_Side Control Time,R__Round5_TIP_Standing Time,winby,winner
0,1,1,23.0,182.0,Trento Italy,2783,Mezzocorona Italy,Carlos Diego Ferreira,84,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,blue
1,0,0,32.0,175.0,"Careiro da Várzea, Amazonas Brazil",2208,"Pharr, Texas USA",Takanori Gomi,70,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB,red
2,2,0,38.0,172.0,Kanagawa Japan,721,Tokyo Japan,Brandon Moreno,70,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO/TKO,blue
3,0,0,23.0,170.0,Tijuana Mexico,2825,Tijuana Mexico,Elizabeth Phillips,56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB,red
4,3,1,30.0,167.0,"Spokane, WA USA",2260,"Spokane, WA USA",Reza Madadi,61,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,blue


In [6]:
ufc.count()

BPrev                              1476
BStreak                            1476
B_Age                              1473
B_Height                           1475
B_HomeTown                         1470
                                   ... 
R__Round5_TIP_Neutral Time          144
R__Round5_TIP_Side Control Time     144
R__Round5_TIP_Standing Time         144
winby                              1460
winner                             1476
Length: 895, dtype: int64

In [7]:
ufc = ufc[ufc.winner != 'no contest']
ufc = ufc[ufc.winner != 'draw']
#ufc_zero = ufc.fillna(0)
ufc_zero = ufc.copy()
ufc_zero.head()

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,B__Round1_Grappling_Reversals_Landed,...,R__Round5_TIP_Ground Time,R__Round5_TIP_Guard Control Time,R__Round5_TIP_Half Guard Control Time,R__Round5_TIP_Misc. Ground Control Time,R__Round5_TIP_Mount Control Time,R__Round5_TIP_Neutral Time,R__Round5_TIP_Side Control Time,R__Round5_TIP_Standing Time,winby,winner
0,1,1,23.0,182.0,Trento Italy,2783,Mezzocorona Italy,Carlos Diego Ferreira,84,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,blue
1,0,0,32.0,175.0,"Careiro da Várzea, Amazonas Brazil",2208,"Pharr, Texas USA",Takanori Gomi,70,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB,red
2,2,0,38.0,172.0,Kanagawa Japan,721,Tokyo Japan,Brandon Moreno,70,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO/TKO,blue
3,0,0,23.0,170.0,Tijuana Mexico,2825,Tijuana Mexico,Elizabeth Phillips,56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB,red
4,3,1,30.0,167.0,"Spokane, WA USA",2260,"Spokane, WA USA",Reza Madadi,61,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,blue


In [8]:
idealcol = ['BPrev','BStreak','B_Age','B_Height','B_Weight','RPrev','RStreak','R_Age','R_Height','R_Weight','winner']
ufc_filtered = ufc_zero[idealcol].copy()

In [9]:
#ufc_filtered["winner"]= ufc_filtered["winner"].replace("red", 0)
#ufc_filtered["winner"]= ufc_filtered["winner"].replace("blue", 1)
#ufc_filtered

In [10]:
ufc_filtered.count()

BPrev       1450
BStreak     1450
B_Age       1447
B_Height    1449
B_Weight    1450
RPrev       1450
RStreak     1450
R_Age       1449
R_Height    1450
R_Weight    1450
winner      1450
dtype: int64

In [11]:
ufc_filtered['BPrev'] = ufc_filtered['BPrev'].fillna(np.mean(ufc_filtered['BPrev']))
ufc_filtered['BStreak'] = ufc_filtered['BStreak'].fillna(np.mean(ufc_filtered['BStreak']))
ufc_filtered['B_Age'] = ufc_filtered['B_Age'].fillna(np.mean(ufc_filtered['B_Age']))
ufc_filtered['B_Height'] = ufc_filtered['B_Height'].fillna(np.mean(ufc_filtered['B_Height']))
ufc_filtered['B_Weight'] = ufc_filtered['B_Weight'].fillna(np.mean(ufc_filtered['B_Weight']))
ufc_filtered['RPrev'] = ufc_filtered['RPrev'].fillna(np.mean(ufc_filtered['RPrev']))
ufc_filtered['RStreak'] = ufc_filtered['RStreak'].fillna(np.mean(ufc_filtered['RStreak']))
ufc_filtered['R_Age'] = ufc_filtered['R_Age'].fillna(np.mean(ufc_filtered['R_Age']))
ufc_filtered['R_Height'] = ufc_filtered['R_Height'].fillna(np.mean(ufc_filtered['R_Height']))
ufc_filtered['R_Weight'] = ufc_filtered['R_Weight'].fillna(np.mean(ufc_filtered['R_Weight']))

In [12]:
ufc_filtered.count()

BPrev       1450
BStreak     1450
B_Age       1450
B_Height    1450
B_Weight    1450
RPrev       1450
RStreak     1450
R_Age       1450
R_Height    1450
R_Weight    1450
winner      1450
dtype: int64

In [13]:
feature_cols2 = ['BPrev','BStreak','B_Age','B_Height','B_Weight','RPrev','RStreak','R_Age','R_Height','R_Weight']
X = ufc_filtered[feature_cols2] # Features

In [14]:
X_scaler = StandardScaler().fit(X)

In [15]:
X_train_scaled = X_scaler.transform(X)

In [16]:
X_test_scaled = X_scaler.transform(X)

In [17]:
ynew = loaded_model.predict(X_test_scaled)

In [18]:
np.round(ynew)

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [19]:
print(ynew)

[[0.3886048  0.6113952 ]
 [0.41066715 0.5893329 ]
 [0.18427862 0.81572145]
 ...
 [0.5764103  0.42358965]
 [0.49095225 0.5090477 ]
 [0.57369417 0.42630583]]
